In [56]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.

This is by no means production code.
"""
# built-in imports
import re
from json import dump
from collections import defaultdict
import random
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import pandas as pd
import csv
import os
import re
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql import functions as F


spark = (
    SparkSession.builder.appName("Assignment_2")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)
pd.options.display.float_format = "{:,.4f}".format

OUTPUT_DIR = "../data/raw/rent/"
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
NUM_PROP = 2500

ConnectionRefusedError: [Errno 111] Connection refused

In [2]:
def write_file(output_dir, file_name, content, mod="w"):
    """
    write given content to local file at: {output_dir}{file_name} with mode: {mod}
    output_dir: the String output folder directory, automatically create if not exist
    file_name: the String file name used to save file
    content: expecting objects which can be written to file with open function
    mod: String of writing mode code used in writing file
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    try:
        with open(f"{output_dir}{file_name}", mod) as f:
            f.write(content)
    except Exception as e:
        print(f"****** Writing file failure: {output_dir}{file_name}")
        print(e)
        print("******")
        return False
    return True



Retrieve post code

In [3]:
### Pull SA2 match table
url = "https://www.matthewproctor.com/Content/\
postcodes/australian_postcodes.csv"
response = requests.get(url, headers=headers)
write_file(f"{OUTPUT_DIR}", "post.csv", response.text)

# Select victoria data then save
vic_df = pd.read_csv(f"{OUTPUT_DIR}post.csv")
post_codes = vic_df.loc[vic_df["state"]=="VIC"]["postcode"].tolist()
post_codes = list(dict.fromkeys(post_codes))
print(len(post_codes))
print(post_codes[:10])


747
[3000, 3001, 3002, 3003, 3004, 3005, 3006, 3008, 3010, 3011]


Retrieve rent property urls

In [37]:
# /rent/melbourne-region-vic/?sort=price-desc&page=
# https://www.domain.com.au/rent/?postcode=3053&sort=default-desc

# constants
BASE_URL = "https://www.domain.com.au"

if not os.path.exists(f"{OUTPUT_DIR}url/"):
    os.makedirs(f"{OUTPUT_DIR}url/")


# begin code
url_links = []
max_page = 50
code_count = 675
file_size = 75
total_code = len(post_codes)
# generate list of urls to visit
while(code_count < len(post_codes)):
    post_code = post_codes[code_count]
    page_count = 0
    
    try:
        for page in range(1, max_page):
            url = BASE_URL + f"/rent/?postcode={post_code}&sort=default-desc&page={page}"
            #bs_object = BeautifulSoup(urlopen(url), "lxml")
            bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")
            # find the unordered list (ul) elements which are the results, then
            # find all href (a) tags that are from the base_url website.
            index_links = bs_object \
                .find(
                    "ul",
                    {"data-testid": "results"}
                ) \
                .findAll(
                    "a",
                    href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
                )
            for link in index_links:
                # if its a property address, add it to the list
                if 'address' in link['class']:
                    url_links.append(link['href'])
            page_count += 1
    except Exception as e:
        print(f"{code_count}: {page_count} pages in post code: {post_code}")

        code_count += 1
        # output to example json in data/raw/
        if code_count%file_size == 0:
            print(f"File writed: {OUTPUT_DIR}url/url_part_{code_count//file_size}.csv")
            with open(f"{OUTPUT_DIR}url/url_part_{code_count//file_size}.csv", "w") as f:
                write = csv.writer(f)
                write.writerow(["url"])
                write.writerows([[i] for i in url_links])
            url_links = []
        
        continue
    
    code_count += 1
    # output to example json in data/raw/
    if code_count%file_size == 0:
        print(f"File writed: {OUTPUT_DIR}url/url_part_{code_count//file_size}.csv")
        with open(f"{OUTPUT_DIR}url/url_part_{code_count//file_size}.csv", "w") as f:
            write = csv.writer(f)
            write.writerow(["url"])
            write.writerows([[i] for i in url_links])
        url_links = []
        

# save urls
if code_count%file_size != 0:
    print(f"File writed: {OUTPUT_DIR}url/url_part_{code_count//file_size}.csv")
    with open(f"{OUTPUT_DIR}url/url_part_{code_count//file_size}.csv", "w") as f:
        write = csv.writer(f)
        write.writerow(["url"])
        write.writerows([[i] for i in url_links])
    url_links = []


450: 0 pages in post code: 3599
451: 0 pages in post code: 3607
452: 0 pages in post code: 3608
453: 0 pages in post code: 3610
454: 0 pages in post code: 3612
455: 0 pages in post code: 3614
456: 1 pages in post code: 3616
457: 0 pages in post code: 3617
458: 1 pages in post code: 3618
459: 0 pages in post code: 3619
460: 1 pages in post code: 3620
461: 1 pages in post code: 3621
462: 0 pages in post code: 3622
463: 0 pages in post code: 3623
464: 0 pages in post code: 3624
465: 1 pages in post code: 3629
466: 4 pages in post code: 3630
467: 1 pages in post code: 3631
468: 0 pages in post code: 3632
469: 0 pages in post code: 3633
470: 1 pages in post code: 3634
471: 0 pages in post code: 3635
472: 0 pages in post code: 3636
473: 0 pages in post code: 3637
474: 0 pages in post code: 3638
475: 0 pages in post code: 3639
476: 0 pages in post code: 3640
477: 1 pages in post code: 3641
478: 0 pages in post code: 3643
479: 1 pages in post code: 3644
480: 0 pages in post code: 3646
481: 0 p

Sample urls

In [4]:
# read urls
url_links = []
for num in range(1,10):
    with open(f"{OUTPUT_DIR}url/url_part_{num}.csv", newline='\n') as f:
        reader = csv.reader(f)
        cur_links = list(reader)
        cur_links = [i[0] for i in cur_links][1:]
        cur_links = list(dict.fromkeys(cur_links))
    url_links = url_links + cur_links

url_links = list(dict.fromkeys(url_links))

# sample url
url_links = random.Random(1001).sample(url_links, NUM_PROP)
print(len(url_links))
print(url_links[:3])

2500
['https://www.domain.com.au/3-180-williamson-street-bendigo-vic-3550-16061472', 'https://www.domain.com.au/3203-118-kavanagh-street-southbank-vic-3006-14147286', 'https://www.domain.com.au/319-lydiard-street-soldiers-hill-vic-3350-16078714']


retrieve property information

In [6]:

if not os.path.exists(f"../data/raw/rent/raw/"):
    os.makedirs(f"../data/raw/rent/raw/")

# for each url, scrape some basic metadata
part = 0
file_size = 50
count = part*file_size
property_metadata = defaultdict(dict)
for property_url in url_links[part*file_size:len(url_links)]:
    print(count, property_url)
    try:
        #bs_object = BeautifulSoup(urlopen(property_url), "lxml")
        bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

        # looks for the header class to get property name
        property_metadata[property_url]['name'] = bs_object \
            .find("h1", {"class": "css-164r41r"}) \
            .text

        # looks for the div containing a summary title for cost
        property_metadata[property_url]['cost_text'] = bs_object \
            .find("div", {"data-testid": "listing-details__summary-title"}) \
            .text
        # extract coordinates from the hyperlink provided
        # i'll let you figure out what this does :P
        property_metadata[property_url]['coordinates'] = [
            float(coord) for coord in re.findall(
                r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
                bs_object \
                    .find(
                        "a",
                        {"target": "_blank", 'rel': "noopener noreferer"}
                    ) \
                    .attrs['href']
            )[0].split(',')
        ]
        property_metadata[property_url]['rooms'] = [
            re.findall(r'\d\s[A-Za-z]+', feature.text)[0] for feature in bs_object \
                .find("div", {"data-testid": "property-features"}) \
                .findAll("span", {"data-testid": "property-features-text-container"})
        ]
        property_metadata[property_url]['desc'] = re \
            .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
            .strip('</p >')
        count+=1
    except:
        count+=1
        if count%file_size == 0:
            print(f"File writed: ../data/raw/rent/raw/rent_raw_part_{count//file_size}.json")
            with open(f'../data/raw/rent/raw/rent_raw_part_{count//file_size}.json', 'w') as f:
                dump(property_metadata, f)
            property_metadata = defaultdict(dict)
        continue
    
    # output to example json in data/raw/
    if count%file_size == 0:
        print(f"File writed: ../data/raw/rent/raw/rent_raw_part_{count//file_size}.json")
        with open(f'../data/raw/rent/raw/rent_raw_part_{count//file_size}.json', 'w') as f:
            dump(property_metadata, f)
        property_metadata = defaultdict(dict)

if count%file_size != 0:
    int(f"File writed: ../data/raw/rent/raw/rent_raw_part_{count//file_size}.json")
    with open(f'../data/raw/rent/raw/rent_raw_part_{count//file_size}.json', 'w') as f:
        dump(property_metadata, f)
    property_metadata = defaultdict(dict)


0 https://www.domain.com.au/3-180-williamson-street-bendigo-vic-3550-16061472
1 https://www.domain.com.au/3203-118-kavanagh-street-southbank-vic-3006-14147286
2 https://www.domain.com.au/319-lydiard-street-soldiers-hill-vic-3350-16078714
3 https://www.domain.com.au/203-clyde-street-soldiers-hill-vic-3350-16080794
4 https://www.domain.com.au/6-13-laura-street-moonee-ponds-vic-3039-13337996
5 https://www.domain.com.au/14-patterson-street-safety-beach-vic-3936-16041036
6 https://www.domain.com.au/6-gent-street-yarraville-vic-3013-16081495
7 https://www.domain.com.au/7-gamble-street-oakleigh-east-vic-3166-12682900
8 https://www.domain.com.au/3-134-brougham-street-kew-vic-3101-15828390
9 https://www.domain.com.au/62-urquhart-street-castlemaine-vic-3450-16027840
10 https://www.domain.com.au/11-coliban-road-werribee-vic-3030-16081270
11 https://www.domain.com.au/47-neptune-street-richmond-vic-3121-16079374
12 https://www.domain.com.au/1-9-whitehorse-road-moolap-vic-3224-16033941
13 https://ww

In [ ]:
digit_re = re.compile(r"(\d+)")
print()

In [68]:
print(property_df.shape)
print(property_df.reset_index()[["cost_text", "coordinates", "bedroom", "Baths", "Parking", "url"]].head())

(1977, 7)
       cost_text                 coordinates bedroom Baths Parking  \
0  $300 per week   [-36.7684112, 144.290465]       2     1       1   
1        $510.00   [-37.8253868, 144.962001]       1     1       1   
2  $330 per week  [-37.7676914, 144.9157867]       1     1       1   
3  $620 per week  [-38.3125918, 144.9980825]       3     2       1   
4           $560  [-37.8139449, 144.8713814]       3     2       1   

                                                 url  
0  https://www.domain.com.au/3-180-williamson-str...  
1  https://www.domain.com.au/3203-118-kavanagh-st...  
2  https://www.domain.com.au/6-13-laura-street-mo...  
3  https://www.domain.com.au/14-patterson-street-...  
4  https://www.domain.com.au/6-gent-street-yarrav...  


In [77]:


property_df = pd.DataFrame(columns=["index", "cost_text", "coordinates", "bedroom", "Baths", "Parking"])
digit_re = re.compile(r'[0-9\,]+')

for page in range(1, 51):
    # read file / drop wrong data
    cur_df = pd.read_json(f'{OUTPUT_DIR}raw/rent_raw_part_{page}.json')
    cur_df = cur_df.T[["cost_text", "coordinates", "rooms"]]
    cur_df = cur_df.dropna()
    cur_df = cur_df[cur_df["rooms"].map(lambda x: len(x) == 3)]
    cur_df = cur_df[cur_df["cost_text"].map(lambda x: len(digit_re.findall(x)) > 0)]

    # split room data into
    cur_df["bedroom"] = cur_df["rooms"].apply(lambda x: digit_re.findall(x[0])[0])
    cur_df["Baths"] = cur_df["rooms"].apply(lambda x: digit_re.findall(x[1])[0])
    cur_df["Parking"] = cur_df["rooms"].apply(lambda x: digit_re.findall(x[2])[0])
    cur_df = cur_df[["cost_text", "coordinates", "bedroom", "Baths", "Parking"]]
    cur_df = cur_df.reset_index().rename({"index": "url", "cost_text": "rent"}, axis=1)

    property_df = pd.concat([property_df, cur_df], axis=0)

property_df = property_df.reset_index()[["rent", "coordinates", "bedroom", "Baths", "Parking", "url"]]
property_df = property_df.dropna().drop_duplicates(subset=['url'])
print(property_df.shape)
print(property_df.tail())

property_df.to_csv(f"{OUTPUT_DIR}rent.csv")

(1924, 6)
               rent                 coordinates bedroom Baths Parking  \
1919    $370 weekly  [-37.8861973, 145.0609476]       1     1       1   
1920        $395 pw  [-37.8623071, 145.0000409]       2     1       1   
1921  $450 per week   [-36.139716, 146.8869266]       3     2       2   
1922        $485.00  [-37.8017087, 145.0351102]       2     2       1   
1923           $650  [-37.8846377, 145.2454077]       5     2       2   

                                                    url  
1919  https://www.domain.com.au/3-22-chestnut-street...  
1920  https://www.domain.com.au/7-154-alma-road-st-k...  
1921  https://www.domain.com.au/16-castleton-street-...  
1922  https://www.domain.com.au/g04-48-pakington-str...  
1923  https://www.domain.com.au/7-foster-crescent-kn...  
